# DataSet

In [1]:
import pandas as pd

## Get Data from other Notebooks

In [2]:
%store -r p_Log
%store -r p_weatherLog

## Variables

Variable | Description
:------------- |:-------------
Date          |   29 non-null object
Value         |   Pump Duration (predicted variable)29 non-null float64
Time          |   29 non-null object
DateTime_x    |   29 non-null object
DateTime_y    |   29 non-null object
tre200b0      |   Lufttemperatur 2 m über Boden; Stundenmittel 24h ago °C (tre200b0)
ure200b0      |   Relative Luftfeuchtigkeit 2 m über Boden; Stundenmittel 24h ago % (ure200b0)
rre150b0      |   Niederschlag; Stundensumme 24h ago mm (rre150b0)
sre000b0      |   Sonnenscheindauer; Stundensumme 24h ago min (sre000b0)
fu3010b0      |   Windgeschwindigkeit skalar; Stundenmittel 24h ago km/h (fu3010b0)
prestab0      |   24h-Mean Luftdruck auf Stationshöhe (QFE); Stundenmittel (from -48h to -24h) hPa (prestab0)
tre200b0_mean |   24h-Mean Lufttemperatur 2 m über Boden; Stundenmittel (from -48h to -24h) °C (tre200b0)
ure200b0_mean |  24h-Mean Relative Luftfeuchtigkeit 2 m über Boden; Stundenmittel (from -48h to -24h) % (ure200b0)
rre150b0_mean  |   24h-Summe Niederschlag; Stundenmittel (from -48h to -24h) mm (rre150b0)
sre000b0_mean  |  24h-Summe Sonnenscheindauer; Stundenmittel (from -48h to -24h) min (sre000b0)
fu3010b0_mean |   24h-Mean Windgeschwindigkeit skalar; Stundenmittel (from -48h to -24h) km/h (fu3010b0)
prestab0_mean |   24h-Mean Luftdruck auf Stationshöhe (QFE); Stundenmittel (from -48h to -24h) hPa (prestab0)
volumelst72h  |  Pump Duration in the last 72 hours

In [20]:
# We want to merge pump log with weather data was 24 ago
# So, add one day to weather data and merge by "DateTime"

#p_weatherLog.index += pd.Timedelta(days=1) "work with actual weather

p_DataSet = p_Log
p_DataSet = p_DataSet.merge(p_weatherLog, 
                            left_on="DateTime", how="left", left_index = False, right_index = True)
#print(p_DataSet)

# Add means and sums
p_weatherLogRolling = p_weatherLog.rolling(24)
p_DataSet = p_DataSet.merge(pd.DataFrame(p_weatherLogRolling.tre200b0.mean()),
                            how="left", left_index = True, right_index = True, suffixes = ["", "_mean"] )
p_DataSet = p_DataSet.merge(pd.DataFrame(p_weatherLogRolling.ure200b0.mean()),
                            how="left", left_index = True, right_index = True, suffixes = ["", "_mean"] )
p_DataSet = p_DataSet.merge(pd.DataFrame(p_weatherLogRolling.rre150b0.mean()),
                            how="left", left_index = True, right_index = True, suffixes = ["", "_mean"] )
p_DataSet = p_DataSet.merge(pd.DataFrame(p_weatherLogRolling.sre000b0.mean()),
                            how="left", left_index = True, right_index = True, suffixes = ["", "_mean"] )
p_DataSet = p_DataSet.merge(pd.DataFrame(p_weatherLogRolling.fu3010b0.mean()),
                            how="left", left_index = True, right_index = True, suffixes = ["", "_mean"] )
p_DataSet = p_DataSet.merge(pd.DataFrame(p_weatherLogRolling.prestab0.mean()),
                            how="left", left_index = True, right_index = True, suffixes = ["", "_mean"] )

p_Log_rolling = p_Log.rolling(4) # 3 Days + Actual Day
p_DataSet = p_DataSet.merge(pd.DataFrame(p_Log_rolling.Value.sum()),
                            how="left", left_index = True, right_index = True, suffixes = ["", "_sum"] )
p_DataSet.Value_sum = p_DataSet.Value_sum - p_DataSet.Value # Substract today's Value to get sum of last 3 days

p_DataSet = p_DataSet.fillna(value = 0)

## Drop no longer needed columns

In [21]:
p_DataSet = p_DataSet.drop(['DateTime_x', 'DateTime_y', 'Time'], axis = 1)

p_DataSet.head()

,Value,tre200b0,ure200b0,rre150b0,sre000b0,fu3010b0,prestab0,tre200b0_mean,ure200b0_mean,rre150b0_mean,sre000b0_mean,fu3010b0_mean,prestab0_mean,Value_sum
Date,,,,,,,,,,,,,,
2014-05-27,960.0,12.4,89.0,0.0,0.0,7.2,958.2,15.733333,69.400000,5.030698e-16,12.750000,8.937500,957.695833,0.0
2014-05-28,480.0,15.8,62.8,0.0,0.0,0.0,956.0,12.445833,83.887500,3.958333e-01,0.291667,11.583333,957.258333,0.0
2014-06-01,960.0,5.8,62.6,0.0,0.0,12.2,957.5,3.600000,87.266667,3.291667e-01,3.750000,6.595833,954.495833,0.0
2014-06-02,12.0,5.4,64.5,0.0,0.0,4.7,948.2,2.495833,90.637500,2.083333e-02,0.041667,6.779167,947.650000,2400.0
2014-06-05,480.0,17.0,43.3,0.0,0.0,16.2,958.9,11.329167,66.387500,5.065393e-16,31.666667,5.066667,959.604167,1452.0


## Save DataSet into CSV File

In [23]:
p_DataSet.to_csv("data/DataSet.csv")